Public score: 0.83787,

Private score: 0.83763.


Ref:

Academic Success| LGBM https://www.kaggle.com/code/igorvolianiuk/academic-success-lgbm
Academic Dataset LightGBM [0.83934] https://www.kaggle.com/code/aspillai/academic-dataset-lightgbm-0-83934#Read-Source-Files
PS4E6 | Academic Success Classification | Ensemble https://www.kaggle.com/code/arunklenin/ps4e6-academic-success-classification-ensemble
📉Droput/Graduate📉Analysis😃 https://www.kaggle.com/code/satyaprakashshukl/droput-graduate-analysis#Thank-u-for-Lgbm-paramter--:)-https://www.kaggle.com/code/aspillai/academic-dataset-lightgbm-0-83767

# Load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import optuna
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e6/train.csv')
original = pd.read_csv('/kaggle/input/students-dropout-and-academic-success/data.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv')

In [3]:
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

correlation_matrix = pd.concat([X_train, y_train], axis=1).corr()
plt.figure(figsize=(20, 18))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [4]:
original[train.columns] = original.iloc[:,0].str.split(';', expand=True)
original.drop(columns=original.columns[0], axis=1, inplace=True)
original = pd.concat([original.drop(columns=['Target']).astype('float64'), original['Target']], axis=1)

# Feature Engineering

In [5]:
def add_features(df):    
    
    #df["Family qualification"] = (df["Mother's qualification"] + df["Father's qualification"]) / 2
    #
    #df["Curricular units 1st sem"] = df["Curricular units 1st sem (credited)"] + df["Curricular units 1st sem (enrolled)"] + df["Curricular units 1st sem (evaluations)"] \
    #                               + df["Curricular units 1st sem (approved)"] + df["Curricular units 1st sem (without evaluations)"]
    #df["Curricular units 2nd sem"] = df["Curricular units 2nd sem (credited)"] + df["Curricular units 2nd sem (enrolled)"] + df["Curricular units 2nd sem (evaluations)"] \
    #                               + df["Curricular units 2nd sem (approved)"] + df["Curricular units 2nd sem (without evaluations)"]
    #df["Curricular units (approved)"] = (df["Curricular units 1st sem (approved)"] + df["Curricular units 2nd sem (approved)"]) / 2
    #df["Curricular units sem (grade)"] = (df["Curricular units 1st sem (grade)"] + df["Curricular units 2nd sem (grade)"]) / 2
    #df["Curricular units 1st sem approved_ratio"] = df["Curricular units 1st sem (approved)"] / (df["Curricular units 1st sem (enrolled)"] + 0.00001)
    #
    df["Curricular units 2nd sem approved_ratio"] = df["Curricular units 2nd sem (approved)"] / (df["Curricular units 2nd sem (enrolled)"] + 0.00001)
    
    return df

train = add_features(train)
original = add_features(original)
test = add_features(test)

In [6]:
le = LabelEncoder()

X_train = train.drop(['Target'], axis=1)
y_train = train['Target']
le.fit(y_train)
y_train = pd.Series(le.transform(y_train))

X_original = original.drop(['Target'], axis=1)
y_original = original['Target']
y_original = pd.Series(le.transform(y_original))

X_test = test

scaler = MinMaxScaler()

scaler.fit(X_train)
scaler.fit(pd.concat([X_train, X_original], axis=0))

X_test = scaler.transform(X_test)

# Fine tuning model

Earlystopping in Optuna: https://stackoverflow.com/questions/74303961/custom-eval-metric-using-early-stopping-in-lgbm-sklearn-api-and-optuna

In [7]:
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'max_depth': trial.suggest_int('max_depth', 2, 13),
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 1.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),    
    }
    
    cv = StratifiedKFold(5, shuffle=True, random_state=0)
    cv_splits = cv.split(X_train, y_train)
    
    val_scores = list()
    
    model = LGBMClassifier(**params, random_state=0, objective='multiclass', device='gpu', verbosity = -1, boosting_type='gbdt')
    
    for train_idx, val_idx in cv_splits:
        X_train_fold = pd.concat([X_train.iloc[train_idx], X_original], axis=0)
        y_train_fold = pd.concat([y_train.iloc[train_idx], y_original], axis=0)
        X_val_fold, y_val_fold = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
        X_train_fold = scaler.transform(X_train_fold)
        X_val_fold = scaler.transform(X_val_fold)
    
        model.fit(X_train_fold, y_train_fold)
        
        y_val_pred = model.predict_proba(X_val_fold)
    
        val_acc= accuracy_score(y_val_fold, np.argmax(y_val_pred, axis=1))
        val_scores.append(val_acc)
    
    return np.mean(val_scores)

sqlite_db = "sqlite:///lgbm.db"
study_name = "lgbm"

optimize = False
if optimize:
    study = optuna.create_study(storage=sqlite_db, study_name=study_name, 
                                sampler=TPESampler(n_startup_trials=75, multivariate=True, seed=0),
                                direction="maximize", load_if_exists=True)

    study.optimize(objective, n_trials=200)
    print(f"best optimized accuracy: {study.best_value:0.5f}") # 0.83441
    print(f"best hyperparameters: {study.best_params}") 
    lgbm_params = study.best_params
else:
    lgbm_params = {
        'learning_rate': 0.008001103599297576, 
        'num_leaves': 170, 
        'min_child_samples': 32, 
        'max_depth': 10, 
        'n_estimators': 1388, 
        'reg_alpha': 2.4199819667303768e-09, 
        'reg_lambda': 0.00012558332991287726, 
        'colsample_bytree': 0.518117499610438, 
        'subsample': 0.9647251538655315
    }

# Validation

In [8]:
cv = StratifiedKFold(5, shuffle=True, random_state=0)
cv_splits = cv.split(X_train, y_train)

val_scores = list()
test_preds = np.zeros((len(X_test), 3))
val_preds = np.zeros((len(X_train), 3))

model = LGBMClassifier(**lgbm_params, random_state=0, objective='multiclass', device='gpu', verbosity = -1, boosting_type='gbdt')
#model = LGBMClassifier(objective='multiclass', random_state=0, device='gpu', verbosity = -1, boosting_type='gbdt')

for train_idx, val_idx in cv_splits:
    
    X_train_fold = pd.concat([X_train.iloc[train_idx], X_original], axis=0)
    y_train_fold = pd.concat([y_train.iloc[train_idx], y_original], axis=0)
    X_val_fold, y_val_fold = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    X_train_fold = scaler.transform(X_train_fold)
    X_val_fold = scaler.transform(X_val_fold)
    
    model.fit(X_train_fold, y_train_fold)
    # , eval_set=[(X_val_fold, y_val_fold)], callbacks=[lgb.early_stopping(stopping_rounds=45)]
    
    val_preds[val_idx] = model.predict_proba(X_val_fold)
    
    y_val_pred = model.predict_proba(X_val_fold)
    
    val_acc= accuracy_score(y_val_fold, np.argmax(y_val_pred, axis=1))
    val_scores.append(val_acc)
    
    y_pred = model.predict_proba(X_test)
    test_preds += y_pred / 5
    
print(f'Mean accuracy score: {np.mean(val_scores):.5f}')

1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Mean accuracy score: 0.83446


In [9]:
accuracy_score(y_train, np.argmax(val_preds, axis=1))

0.8344572518884446

# Submission

In [10]:
y_test = np.argmax(test_preds, axis=1)
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e6/sample_submission.csv')
sample_submission['Target'] = le.inverse_transform(y_test)
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Enrolled
4,76522,Enrolled
